In [1]:
import numpy as np
import pandas as pd
import urllib

Scrap neighbourhood info from wikipedia

In [2]:
con=urllib.request.urlopen("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
page=con.read()
content=page.decode("UTF-8")

The page only contains tables so we can extract its data by using table tags

In [3]:
table = content[content.find("<table"):content.find("</table>")+8]

In [4]:
data=pd.read_html(table,header=0)[0]

Beautiful Soup not used here 

we have to select only those boroughs that have an assigned name and assign borough name to the neighbourhood

In [6]:
data = data[data.Borough != "Not assigned"]
data.Neighbourhood[data.Neighbourhood == "Not assigned"] = data.Borough[data.Neighbourhood == "Not assigned"]
data.head()

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights


Now we will define a function that takes a series of names and combines them with commas between them

In [7]:
def concatenate_neighbourhood(x):
    cadena = ""
    for i in range(len(x)-1):
        cadena = cadena + x.iloc[i] + ", "
    cadena += x.iloc[-1]
    return cadena

We will also have a function that checks the postal code is completely contained within the  borough. 

In [8]:

def select_Borough(x):
    ref = x.iloc[0]
    for i in range(1, len(x)):
        if ref != x.iloc[i]:
            for i in x:
                print(x)
            raise Exception("Postcode comprises two Boroughs")
    return ref

In [11]:
pdata = data.groupby(["Postcode"]).agg({"Borough": lambda x: select_Borough(x),
                                 "Neighbourhood": lambda x: concatenate_neighbourhood(x)})
pdata.head(11)

,Borough,Neighbourhood
Postcode,,
M1B,Scarborough,"Rouge, Malvern"
M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
M1E,Scarborough,"Guildwood, Morningside, West Hill"
M1G,Scarborough,Woburn
M1H,Scarborough,Cedarbrae
M1J,Scarborough,Scarborough Village
M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park"
M1L,Scarborough,"Clairlea, Golden Mile, Oakridge"
M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West"


In [10]:
pdata.shape

(103, 2)